# SRA ChIP-Seq Data

We are wanting pol II ChIP-Seq data in the testis. There is one dataset in GEO, but I want to look in SRA if there are any additional datasets. This notebook uses my MongoDB from ncbi_remap for easy querying in SRA. 

In [36]:
# %load defaults.py
# Imports
import os
import sys
from pathlib import Path
import re

from IPython.display import display, HTML, Markdown
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Project level imports
from larval_gonad.notebook import Nb

In [5]:
# Setup notebook
nbconfig = Nb.setup_notebook()

last updated: 2018-07-17 
Git hash: 09c6fb287994a19d25111e9a64bdc2fd8dcb68b3


In [3]:
from pymongo import MongoClient
host = 'localhost'
mongoClient = MongoClient(host=host, port=27017)
db = mongoClient['sra']
ncbi = db['ncbi']

In [7]:
chipdat = [x['_id'] for x in ncbi.aggregate([
    {
        '$match': {
            'sra.experiment.library_strategy': 'ChIP-Seq'
        }
    },
    {
        '$project': {
            '_id': 1
        }
    }
])]

In [43]:
aggr = ncbi.aggregate([
    {
        '$match': {
            '_id': {'$in': chipdat}
        }
    },
    {
        '$unwind': {
            'path': '$sra.sample.attributes'
        }
    },
    {
        '$project': {
            'title': '$sra.sample.title',
            'attr_name': '$sra.sample.attributes.name',
            'attr_val': '$sra.sample.attributes.value',
        }
    },
    {
        '$group': {
            '_id': '$_id',
            'title': {'$first': '$title'},
            'vals': {'$addToSet': '$attr_val'}
        }
    },
])

res = []
for i in aggr:
    _title = i.get('title', False)
    if _title:
        attrs = [_title]
    else:
        attrs = []
    attrs.extend(i['vals'])
    string = ' '.join(attrs).lower()
    if re.search(r'test[ie]s', string):
        if re.search(r'pol', string):
            res.append((i['_id'], string.lower()))

In [44]:
res

[('SRX2748311',
  'ez_polii-ga0200_chipseq testis e(z)61 bam double mutant rna pol ii (abcam, ab5408) testis, e(z)61 bam double mutant, polii chip'),
 ('SRX017857', 'bam polii chip-seq ab5408 abcam polii bam testis')]